In [1]:
from keras.models import load_model
model = load_model('enzyme_rnn_v3.h5')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [2]:
import pandas as pd
inputs=pd.read_csv('inputs.csv')
gt=pd.read_csv('gt.csv')
batch=inputs.join(gt)
batch.columns=['Amino acid', 'Class']
x=batch['Amino acid']
y=batch['Class']

In [6]:
#Data preprocessing
import urllib.request
import warnings
import numpy as np
from Bio import PDB
warnings.filterwarnings("ignore")
counter = 0
sequences = np.array([])
failed = np.array([])
short = np.array([])
for x in batch['Amino acid']:
    counter += 1
    urllib.request.urlretrieve("http://files.rcsb.org/download/" + x +".pdb", x + ".pdb")
    parser=PDB.PDBParser()
    struct=parser.get_structure(x,x + '.pdb')
    atom=list(struct.get_atoms())[0]
    pp=PDB.PPBuilder()
    polypeptide=pp.build_peptides(struct)
    if len(polypeptide) == 0:
        failed = np.append(failed, counter-1)
        #print("Amino acid " + x + " not parsed correctly.")
    elif len(polypeptide) > 0:
        polypeptide=polypeptide[0]
        sequence=polypeptide.get_sequence()
        if len(str(sequence)) < 50:
            short = np.append(short, counter-1)
            #print("Amino acid " + x + "too short.")
        else:
            sequences = np.append(sequences, str(sequence))
    if counter % 50 == 0:
        print(counter, "amino acids processed.")

50 amino acids processed.
100 amino acids processed.
150 amino acids processed.
200 amino acids processed.
250 amino acids processed.
300 amino acids processed.
350 amino acids processed.
400 amino acids processed.
450 amino acids processed.


In [7]:
def int_concatenate(sequence):
    vals = []
    for a in sequence:
        seq = [ord(x) for x in a]
        seq = [seq]
        vals += seq
    return vals 
intvalues = int_concatenate(sequences)
dropped = batch
fail = np.append(failed, short)
fail = np.sort(fail)
fail = np.flip(fail)
for x in fail:
    dropped = dropped.drop(dropped.index[int(x)], axis=0)
dropped['Sequence'] = sequences
x = np.array(intvalues)

In [8]:
#Predicting, and adjusting results to integer label values
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
encoder=OneHotEncoder()
encoder.fit(np.array(list(x[0])).reshape(-1,1)) 
x=np.array([encoder.transform(np.array(list(item[:50])).reshape(-1,1)).toarray() for item in x])
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)

In [120]:
y_classes

array([2, 3, 3, 3, 1, 1, 2, 5, 2, 4, 4, 1, 3, 2, 4, 3, 3, 3, 3, 1, 3, 1,
       2, 1, 2, 1, 2, 3, 1, 2, 1, 2, 3, 1, 2, 2, 2, 1, 3, 1, 3, 3, 2, 1,
       3, 4, 1, 1, 2, 1, 3, 3, 2, 3, 2, 3, 4, 1, 1, 1, 5, 3, 2, 2, 2, 3,
       1, 1, 1, 3, 2, 1, 3, 3, 1, 3, 3, 3, 6, 1, 4, 1, 1, 3, 2, 2, 3, 2,
       3, 3, 1, 3, 1, 2, 2, 4, 2, 3, 1, 1, 1, 3, 1, 3, 2, 5, 4, 1, 3, 3,
       1, 2, 3, 3, 3, 2, 2, 1, 3, 3, 3, 2, 1, 3, 2, 1, 1, 1, 1, 3, 6, 1,
       3, 1, 1, 3, 2, 3, 2, 1, 3, 3, 1, 1, 3, 3, 2, 5, 2, 1, 1, 1, 3, 1,
       3, 1, 4, 2, 2, 2, 3, 1, 1, 3, 3, 1, 3, 2, 3, 2, 3, 2, 1, 1, 3, 2,
       2, 3, 1, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 3, 5, 1, 3, 3, 2, 3, 1,
       1, 2, 1, 3, 3, 2, 2, 3, 3, 3, 1, 2, 1, 3, 2, 2, 4, 1, 3, 3, 1, 2,
       2, 1, 1, 2, 3, 2, 4, 1, 3, 2, 2, 4, 1, 3, 1, 5, 2, 3, 3, 1, 2, 5,
       3, 3, 2, 4, 6, 1, 3, 1, 1, 1, 3, 1, 3, 2, 2, 3, 2, 1, 1, 3, 3, 3,
       1, 3, 2, 3, 1, 3, 1, 1, 6, 1, 2, 3, 2, 2, 2, 1, 2, 2, 2, 6, 1, 2,
       1, 3, 3, 3, 1, 2, 3, 3, 2, 3, 3, 3, 3, 2, 3,